In [57]:
import os

# Change this to your working directory
os.chdir(r'C:\project')

**Importer les bibliothèques nécessaires**

In [58]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

**Chargement des données**

In [59]:
# Lecture du fichier nettoyé
df = pd.read_csv("data/avito_cars_clean.csv")

# Afficher les cinq premières lignes des données
df.head()

,annee,boite,carburant,kilometrage,marque,modele,nombre_portres,premiere_main,puissance_fiscale,etat,prix
0,2023,Automatique,Essence,22499.5,Alfa Romeo,Tonale,5,Oui,8,Excellent,306000
1,2018,Manuelle,Diesel,104999.5,Dacia,Duster,5,Oui,6,Excellent,155000
2,2023,Automatique,Diesel,67499.5,Hyundai,Accent,5,Oui,6,Excellent,179000
3,2023,Automatique,Diesel,22499.5,Peugeot,5008,5,Oui,8,Excellent,349000
4,2021,Automatique,Diesel,77499.5,Kia,Sportage,5,Oui,12,Très bon,685000


**Examen des données**

In [60]:
print(f"Dimensions des données : {df.shape}")
print(f"Statistiques des prix : Min={df['prix'].min()}, Max={df['prix'].max()}, Moyenne={df['prix'].mean()}")
print(f"Colonnes : {df.columns.tolist()}")

Dimensions des données : (68415, 11)
Statistiques des prix : Min=5200, Max=1000000, Moyenne=138682.62904333844
Colonnes : ['annee', 'boite', 'carburant', 'kilometrage', 'marque', 'modele', 'nombre_portres', 'premiere_main', 'puissance_fiscale', 'etat', 'prix']


**Vérifier et nettoyer les données**

In [61]:
print(df.info())
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68415 entries, 0 to 68414
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   annee              68415 non-null  int64  
 1   boite              68415 non-null  object 
 2   carburant          68415 non-null  object 
 3   kilometrage        68415 non-null  float64
 4   marque             68415 non-null  object 
 5   modele             68415 non-null  object 
 6   nombre_portres     68415 non-null  int64  
 7   premiere_main      68415 non-null  object 
 8   puissance_fiscale  68415 non-null  int64  
 9   etat               68415 non-null  object 
 10  prix               68415 non-null  int64  
dtypes: float64(1), int64(4), object(6)
memory usage: 5.7+ MB
None
annee                0
boite                0
carburant            0
kilometrage          0
marque               0
modele               0
nombre_portres       0
premiere_main        0
puissance_fiscal

***Cela signifie que les données sont bien structurées et prêtes pour l'analyse, et qu’il n’y a pas de données manquantes à traiter.***

**Gestion des valeurs aberrantes (AVANT la division des données)**

In [62]:
Q1 = df['prix'].quantile(0.25)
Q3 = df['prix'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
    
print(f"Filtrage des prix entre {lower_bound} et {upper_bound}")
df_filtered = df[(df['prix'] >= lower_bound) & (df['prix'] <= upper_bound)]
print(f"Valeurs aberrantes supprimées : {df.shape[0] - df_filtered.shape[0]}")

Filtrage des prix entre -95505.0 et 332503.0
Valeurs aberrantes supprimées : 4041


**Sélection des caractéristiques et prétraitement**

In [63]:
# Garder les caractéristiques numériques et catégorielles
features = ['annee', 'kilometrage', 'puissance_fiscale', 'etat', 'marque', 'modele', 'boite', 'carburant']
target = 'prix'

# Création de X et y
X = df_filtered[features]
y = df_filtered[target]

***XGBoost ne gère pas les textes directement, donc on convertit les colonnes textuelles en chiffres.***

In [64]:
# Encodage des variables catégorielles
label_encoders = {}
categorical_cols = ['etat', 'marque', 'modele', 'boite', 'carburant']
    
for col in categorical_cols:
    if col in X.columns:
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col])
        label_encoders[col] = le

C:\Users\ss\AppData\Local\Temp\ipykernel_23024\1442985565.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = le.fit_transform(X[col])
C:\Users\ss\AppData\Local\Temp\ipykernel_23024\1442985565.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = le.fit_transform(X[col])
C:\Users\ss\AppData\Local\Temp\ipykernel_23024\1442985565.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

**Suppression des lignes avec des valeurs manquantes**

In [65]:
X = X.dropna()
y = y[X.index]

**On va entraîner le modèle sur 80% des données et tester sur 20%.**

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Mise à l'échelle des caractéristiques**

In [67]:
scaler = StandardScaler()
numeric_cols = ['annee', 'kilometrage', 'puissance_fiscale']
X_train_numeric = scaler.fit_transform(X_train[numeric_cols])
X_test_numeric = scaler.transform(X_test[numeric_cols])
    
# Conversion en DataFrame pour faciliter la manipulation
X_train_numeric_df = pd.DataFrame(X_train_numeric, columns=numeric_cols, index=X_train.index)
X_test_numeric_df = pd.DataFrame(X_test_numeric, columns=numeric_cols, index=X_test.index)
    
# Remplacement des colonnes numériques par leurs versions mises à l'échelle
for col in numeric_cols:
    X_train[col] = X_train_numeric_df[col]
    X_test[col] = X_test_numeric_df[col]

**Création et entraînement du modèle XGBoost**

In [68]:
model = xgb.XGBRegressor(
    objective='reg:squarederror', 
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    random_state=42
)
    
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.1, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=100,
             n_jobs=None, num_parallel_tree=None, ...)

**Prédictions avec les données de test**

In [69]:
y_pred = model.predict(X_test)

**Évaluation du modèle**

In [70]:
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
    
print("\nPerformance du modèle XGBoost :")
print(f"Erreur absolue moyenne (MAE) : {mae:.2f}")
print(f"Erreur quadratique moyenne (RMSE) : {rmse:.2f}")
print(f"R² (coefficient de détermination) : {r2:.4f}")


Performance du modèle XGBoost :
Erreur absolue moyenne (MAE) : 16667.01
Erreur quadratique moyenne (RMSE) : 25761.48
R² (coefficient de détermination) : 0.8694


**Visualisation de l'importance des caractéristiques**

In [71]:
importance = model.feature_importances_
feature_names = X_train.columns
    
# Créer un DataFrame pour l'importance des caractéristiques
feature_importance = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importance
}).sort_values(by='Importance', ascending=False)
    
print("\nImportance des caractéristiques :")
print(feature_importance)


Importance des caractéristiques :
             Feature  Importance
6              boite    0.445697
0              annee    0.345303
7          carburant    0.066881
2  puissance_fiscale    0.050541
4             marque    0.035261
5             modele    0.025143
3               etat    0.022121
1        kilometrage    0.009054


**Comparaison des valeurs prédites vs réelles (échantillon)**

In [72]:
comparison = pd.DataFrame({
    'Réel': y_test.values[:20],
    'Prédit': y_pred[:20],
    'Différence': y_test.values[:20] - y_pred[:20]
})
print("\nComparaison des prédictions (20 premiers exemples) :")
print(comparison)


Comparaison des prédictions (20 premiers exemples) :
      Réel         Prédit    Différence
0    65000   70692.960938  -5692.960938
1    58000   69382.132812 -11382.132812
2   165000  150787.421875  14212.578125
3   250000  273937.906250 -23937.906250
4    65000   55724.558594   9275.441406
5    27000   31425.132812  -4425.132812
6    50000   50144.847656   -144.847656
7   148000  147990.812500      9.187500
8   217000  191101.203125  25898.796875
9    85000   85335.945312   -335.945312
10   45000   43471.152344   1528.847656
11  330000  265264.750000  64735.250000
12  135000  117980.132812  17019.867188
13  120000  124517.812500  -4517.812500
14  180000  170170.921875   9829.078125
15   52000   51965.386719     34.613281
16  105000  106813.460938  -1813.460938
17  290000  281164.375000   8835.625000
18   90000  102879.210938 -12879.210938
19   83000   82395.054688    604.945312


In [73]:
import joblib

# Sauvegarder le scaler
joblib.dump(scaler, 'pkl-files/xgboost_scaler.pkl')

# Sauvegarder le modèle
joblib.dump(model, 'pkl-files/xgboost_model.pkl')

# Sauvegarder les colonnes utilisées
joblib.dump(X.columns, 'pkl-files/xgboost_columns.pkl')

print("Le scaler et le modèle de xgboost ont été sauvegardés sous forme de fichiers .pkl.")

Le scaler et le modèle de xgboost ont été sauvegardés sous forme de fichiers .pkl.
